In [ ]:
import open3d as o3d
import numpy as np

# Load mesh
mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/GrateAndCover/gratenew.obj")
mesh.compute_triangle_normals()

# OPTIONAL: Visualize to pick a crop manually
# o3d.visualization.draw_geometries([mesh])

# -------------------
# CHRIS: UPDATE THESE BOUNDS BASED ON VISUAL SELECTION
# You can use the bounding box of bend 1 if it's known
crop_min = np.array([50, 0, -100])
crop_max = np.array([190, 170, 100])
# -------------------

crop_box = o3d.geometry.AxisAlignedBoundingBox(crop_min, crop_max)
mesh_crop = mesh.crop(crop_box)

# Visualize cropped mesh (optional sanity check)
# o3d.visualization.draw_geometries([mesh_crop])

# Create raycasting scene
scene = o3d.t.geometry.RaycastingScene()
mesh_id = scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh_crop))

# Use only the XY range of the crop box
min_x, min_y, _ = crop_min
max_x, max_y, _ = crop_max

# Increase resolution for dense sampling
num_x = 2000
num_y = 2000

x_vals = np.linspace(min_x, max_x, num_x)
y_vals = np.linspace(min_y, max_y, num_y)
xx, yy = np.meshgrid(x_vals, y_vals)

origin_z = crop_max[2] + 10  # slightly above surface

origins = np.stack([xx.ravel(), yy.ravel(), np.full(xx.size, origin_z)], axis=1)
directions = np.tile([0, 0, -1], (origins.shape[0], 1))

rays = o3d.core.Tensor(np.hstack((origins, directions)), dtype=o3d.core.Dtype.Float32)
hits = scene.cast_rays(rays)

mask = hits['t_hit'].isfinite()
hit_points = origins[mask.numpy()] + hits['t_hit'][mask].numpy().reshape(-1, 1) * directions[mask.numpy()]

pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(hit_points))
o3d.io.write_point_cloud("sampled_dense_region.ply", pcd)
o3d.visualization.draw_geometries([pcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
print("Mesh bounds:", np.asarray(mesh.get_axis_aligned_bounding_box().get_extent()))
print("Ray origin Z:", origins[0][2])
print("Ray direction:", directions[0])

Mesh bounds: [300.         300.          16.89474106]
Ray origin Z: 26.89474105834961
Ray direction: [ 0  0 -1]


In [50]:
import open3d as o3d
import numpy as np

# Load mesh
mesh = o3d.io.read_triangle_mesh("C:/Users/chris/Downloads/BendyCAD.STL")
mesh.compute_vertex_normals()

# Step 1: Apply rotation so that the scanned face is up
# You said previously this was: x=290, y=0, z=270
Rx = mesh.get_rotation_matrix_from_axis_angle([np.deg2rad(110), 0, 0])
Ry = mesh.get_rotation_matrix_from_axis_angle([0, np.deg2rad(0), 0])
Rz = mesh.get_rotation_matrix_from_axis_angle([0, 0, np.deg2rad(90)])

# Apply in ZYX order
mesh.rotate(Rx, center=mesh.get_center())
mesh.rotate(Ry, center=mesh.get_center())
mesh.rotate(Rz, center=mesh.get_center())

# Step 2: Top-down camera
o3d.visualization.draw_geometries(
    [mesh])


In [51]:
def raycast_topdown(mesh, spacing=0.2, origin_offset=10.0):
    import open3d as o3d
    import numpy as np

    mesh.compute_triangle_normals()
    scene = o3d.t.geometry.RaycastingScene()
    t_mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    _ = scene.add_triangles(t_mesh)

    # Bounding box (after rotation)
    aabb = mesh.get_axis_aligned_bounding_box()
    min_x, min_y, _ = aabb.min_bound
    max_x, max_y, _ = aabb.max_bound
    width = max_x - min_x
    height = max_y - min_y

    # Ray grid resolution based on spacing
    num_x = int(width / spacing)
    num_y = int(height / spacing)
    x_vals = np.linspace(min_x, max_x, num_x)
    y_vals = np.linspace(min_y, max_y, num_y)
    xx, yy = np.meshgrid(x_vals, y_vals)

    # Place origins above the mesh
    origin_z = aabb.max_bound[2] + origin_offset
    origins = np.stack([xx.ravel(), yy.ravel(), np.full(xx.size, origin_z)], axis=1)
    directions = np.tile([0, 0, -1], (origins.shape[0], 1))  # straight down

    rays = o3d.core.Tensor(np.hstack((origins, directions)), dtype=o3d.core.Dtype.Float32)
    hits = scene.cast_rays(rays)

    mask = hits['t_hit'].isfinite()
    hit_points = origins[mask.numpy()] + hits['t_hit'][mask].numpy().reshape(-1, 1) * directions[mask.numpy()]
    return o3d.geometry.PointCloud(o3d.utility.Vector3dVector(hit_points))


In [52]:
pcd = raycast_topdown(mesh)
o3d.visualization.draw_geometries([mesh, pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
